In [1]:
import tensorflow as tf

In [3]:
mnist = tf.keras.datasets.mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 3s 0us/step


In [5]:
x_train, x_test = x_train/255., x_test / 255.

In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 63us/sample - loss: 0.2950 - accuracy: 0.9144
Epoch 2/5
60000/60000 [==============================] - 3s 46us/sample - loss: 0.1438 - accuracy: 0.9572
Epoch 3/5
60000/60000 [==============================] - 3s 48us/sample - loss: 0.1089 - accuracy: 0.9672
Epoch 4/5
60000/60000 [==============================] - 3s 47us/sample - loss: 0.0903 - accuracy: 0.9723s - loss: 0
Epoch 5/5
60000/60000 [==============================] - 3s 46us/sample - loss: 0.0787 - accuracy: 0.9751


In [9]:
model.evaluate(x_test, y_test, verbose=2)

10000/10000 - 1s - loss: 0.0751 - accuracy: 0.9758


[0.07506956264870242, 0.9758]

# Experts

In [10]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [12]:
x_train = x_train[..., tf.newaxis]

In [13]:
x_test.shape

(10000, 28, 28)

In [14]:
x_test = x_test[..., tf.newaxis]

In [15]:
x_test.shape

(10000, 28, 28, 1)

In [18]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

In [19]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).shuffle(10000).batch(32)

In [21]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        s = self.d1(x)
        return self.d2(x)

In [22]:
model = MyModel()

In [23]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

In [24]:
optimizer = tf.keras.optimizers.Adam()

In [25]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [26]:
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [27]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [28]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [35]:
@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [36]:
EPOCHS = 5
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
    
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
        epoch + 1,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100
    ))

Epoch 1, Loss: 0.0479426234960556, Accuracy: 98.48333740234375, Test Loss: 0.06129706650972366, Test Accuracy: 98.02999877929688
Epoch 2, Loss: 0.03540346771478653, Accuracy: 98.9366683959961, Test Loss: 0.06387746334075928, Test Accuracy: 98.04000091552734
Epoch 3, Loss: 0.02685713954269886, Accuracy: 99.16333770751953, Test Loss: 0.06222997233271599, Test Accuracy: 98.19999694824219
Epoch 4, Loss: 0.0203801728785038, Accuracy: 99.38333892822266, Test Loss: 0.07154785096645355, Test Accuracy: 98.06999969482422
Epoch 5, Loss: 0.015406295657157898, Accuracy: 99.51333618164062, Test Loss: 0.06668094545602798, Test Accuracy: 98.12999725341797
